nurbs2points multiplication dimensions:


In [ ]:
from __future__ import print_function
import numpy
import random
import math
import matplotlib.pyplot as plt
import numpy.random
import datetime
import os

na = numpy.asarray


import math
import random

# import gym
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Normal

from IPython.display import clear_output
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import display

use_cuda = torch.cuda.is_available()
device   = torch.device("cuda" if use_cuda else "cpu")
# device = torch.device("cpu") 
print(device)

In [ ]:
# track utils

def mesh2tessels(cp_vec, cp_wei, unit_dist=None, arbi_pxno=None):
    recur_norm_rank = 16
#     arbi_pxno = 16
#     unit_dist = 5
    pxno_auto = bool(unit_dist)
    recur_stop_u_max_ratio_cond = 0.01

    # data
    nurbs_rank = len(cp_vec)
    coords_mx = cp_vec
    weight_mx = cp_wei

    # curve calculation matrices
    contribution_mx = na(
        [[(-1)**(bernstein_rank+bernstein_pos) 
            * math.comb(bernstein_rank, bernstein_pos) 
            * math.comb(nurbs_rank-1,   bernstein_rank)
        for bernstein_rank in range(nurbs_rank-1, -1, -1)]
        for bernstein_pos  in range(nurbs_rank-1, -1, -1)])
    nominator_mx = contribution_mx @ (coords_mx * weight_mx)
    denominator_mx = contribution_mx @ weight_mx

    # unit lenght no estimation
    if pxno_auto:
        mesh_len = numpy.sum(numpy.sum((coords_mx[1:] - coords_mx[:-1])**2, axis=1) **.5)
        mesh_pxno = int(round(mesh_len / unit_dist))
        u_arr = numpy.linspace(0, 1, mesh_pxno, endpoint=True)
        u_mx = u_arr[:, None] ** numpy.arange(nurbs_rank) # u_no x [u**0 ... u**n]

        tessels = u_mx @ nominator_mx / (u_mx @ denominator_mx)
        distances = (numpy.sum((tessels[1:] - tessels[:-1])**2, axis=1)**.5)[:, None]
        total_length = numpy.sum(distances)
        pxno = max(int(round(total_length / unit_dist)), nurbs_rank)
    else:
        pxno = arbi_pxno
    u_arr = numpy.linspace(0, 1, pxno, endpoint=True)
    u_mx = u_arr[:, None] ** numpy.arange(nurbs_rank) # u_no x [u**0 ... u**n]

    # parameter normalization
    deriv_coeff_mx = na(
        [[ (derivative_pos) * (derivative_pos == primitive_pos + 1)
        for derivative_pos in range(nurbs_rank)]
        for primitive_pos  in range(nurbs_rank)])
    nominator_deriv_mx = deriv_coeff_mx @ nominator_mx
    denominator_deriv_mx = deriv_coeff_mx @ denominator_mx

    for _ in range(recur_norm_rank):
        u_ver = u_arr
        u_arr = na([el for tup in zip(u_arr[:-1], (u_arr[:-1]+u_arr[1:])/2) for el in tup] + [u_arr[-1]])
        u_mx = u_arr[:, None] ** numpy.arange(nurbs_rank)

        deriv_mx = (((u_mx @ nominator_deriv_mx) * (u_mx @ denominator_mx) 
        - (u_mx @ denominator_deriv_mx) * (u_mx @ nominator_mx)) 
        / (u_mx @ denominator_mx)** 2)

        u_drv = (1/(  # Simpson's method
            numpy.sum(deriv_mx[:-2:2]**2, axis=1)**.5 
            + 4*numpy.sum(deriv_mx[1:-1:2]**2, axis=1)**.5 
            + numpy.sum(deriv_mx[2::2]**2, axis=1)**.5 #* numpy.sqrt(u_arr[2::2] - u_arr[:-2:2]) # todo: monzyc razy u - nierowna dystrybucja
        ))
        u_drv = numpy.concatenate(([0], u_drv.cumsum()))

        u_arr = ((u_drv - u_drv[0])/(u_drv[-1] - u_drv[0]))
        u_mx = u_arr[:, None] ** numpy.arange(nurbs_rank)

        if numpy.max(numpy.abs(u_arr[1:-1]/u_ver[1:-1] -1)) < recur_stop_u_max_ratio_cond:
            break
    else:
        print("warning: max no of tesselation normalisation recurrences reached ({})!".format(recur_norm_rank))
        
    pointset = u_mx @ nominator_mx / (u_mx @ denominator_mx)
    deriv_mx = (((u_mx @ nominator_deriv_mx) * (u_mx @ denominator_mx) 
        - (u_mx @ denominator_deriv_mx) * (u_mx @ nominator_mx)) 
        / (u_mx @ denominator_mx)** 2)
    directions = deriv_mx / (numpy.sum(deriv_mx **2, axis=1)**.5)[:, None]
    return pointset, directions


ceil = lambda x: int(x) + (x != int(x))

def race_gen(seed, radius):
    norm_mean = 200
    norm_std = norm_mean*.8
    arc_mean = numpy.pi * .2
    arc_std = numpy.pi * .2
    weight_mean = 1
    weight_std = .3
    numpy.random.seed(seed)

    arc_sum = 2 * numpy.pi
    arcs = numpy.random.normal(arc_mean, arc_std, 2*int(arc_sum/arc_mean))
    arcs = arcs.cumsum()
    arcs = arcs[arcs < arc_sum]

    norms = numpy.random.normal(norm_mean, norm_std, len(arcs))

    genpoints = norms[:, None] * na([numpy.cos(arcs), numpy.sin(arcs)]).T

    # remove points of too low cos
    gp_cos_lim = .2
    for _ in range(16):
        gp_sh_l = numpy.concatenate((genpoints[-1][None], genpoints[:-1]), axis=0)
        gp_sh_r = numpy.concatenate((genpoints[1:], genpoints[0][None]), axis=0)
        coses = (numpy.cross(gp_sh_l-genpoints, gp_sh_r-genpoints, axisa=1, axisb=1) 
                 / ((numpy.sum((gp_sh_l-genpoints)**2, axis=1)**.5) * (numpy.sum((gp_sh_r-genpoints)**2, axis=1)**.5)))
        gp_accepted = numpy.abs(coses) > gp_cos_lim
        genpoints = genpoints[gp_accepted, :]
        if numpy.all(gp_accepted):
            break
    else:
        print("warning: point cos acceptance")

    genpoints_no = len(genpoints)
    genweights = numpy.random.normal(weight_mean, weight_std, genpoints_no)

    curve_rank = 4

    genpoints_expanded = numpy.concatenate((genpoints, genpoints[:(curve_rank-2)]))
    genweights_expanded = numpy.concatenate((genweights, genweights[:(curve_rank-2)]))
    genpoints_eno = len(genpoints_expanded)

    unit_meshes = []
    tessels = []
    directions = []

    track_pxs = []
    track_drs = []

    for umr_cp0ix in range(0, genpoints_no, curve_rank-2):
        unit_mesh_raw = genpoints_expanded[umr_cp0ix:(umr_cp0ix+curve_rank)]
        um_weights_raw = genweights_expanded[umr_cp0ix:(umr_cp0ix+curve_rank)]

        unit_mesh = unit_mesh_raw.copy()
        unit_mesh[0] = numpy.mean(unit_mesh_raw[:2], axis=0)
        unit_mesh[-1] = numpy.mean(unit_mesh_raw[-2:], axis=0)

        um_weights = um_weights_raw[:, None]
    #     print(unit_mesh)

        unit_meshes.append((unit_mesh, um_weights))
        tessels, directions = mesh2tessels(unit_mesh, um_weights, unit_dist=5)

        track_pxs.append(tessels)
        track_drs.append(directions)

#         plt.plot(*tessels.T)

    rad_mean = radius
    rad_std = 0
    rad_min = rad_mean - rad_std
    rad_max = rad_mean + rad_std
    rad_checkpoint = rad_min

    mesh_count = len(track_pxs)

    mesh_sides_cyc = [[], []]

#     fix, ax = plt.subplots(1, 1, figsize=(15,15))

    # side_gen
    for tix in range(mesh_count):
        track_px = track_pxs[tix]
        track_dr = track_drs[tix]

        tessels_no = len(track_px)

        track_width = numpy.clip(
            numpy.random.normal(rad_mean, rad_std, tessels_no), 
            numpy.ones(tessels_no)*rad_min,
            numpy.ones(tessels_no)*rad_max
        )[:,None]

        side_l = track_px + (track_dr[:, ::-1] * [-1, 1] * track_width)
        side_r = track_px + (track_dr[:, ::-1] * [1, -1] * track_width)

    #     print(side_l.shape)

        bbox_l = na((numpy.min(side_l, axis=0), numpy.max(side_l, axis=0)))
        bbox_r = na((numpy.min(side_r, axis=0), numpy.max(side_r, axis=0)))

        mesh_sides_cyc[0].append((side_l, bbox_l, tix, 1))
        mesh_sides_cyc[1].append((side_r, bbox_r, tix, -1))

#         ax.plot(*track_px.T, lw=3)

    #     bbox = bbox_l
    #     ax.plot(*na([bbox[0], (bbox[0][0], bbox[1][1]), bbox[1], (bbox[1][0], bbox[0][1]), bbox[0]]).T, lw=.5)
    #     bbox = bbox_r
    #     ax.plot(*na([bbox[0], (bbox[0][0], bbox[1][1]), bbox[1], (bbox[1][0], bbox[0][1]), bbox[0]]).T, lw=.5)


    # crossection_detection

    mesh_sides = []  # side_id := index
    side_splits = []
    # marginal_side_splits = []

    for primitive_cycle in mesh_sides_cyc:
        side_ix_shift = len(mesh_sides)
        no_sides_in_cycle = len(primitive_cycle)

    #     print(side_ix_shift, len(primitive_cycle))
        spurrious_crossings = []
        for ms_ix, mesh_side in enumerate(primitive_cycle):
            absolute_side_index = side_ix_shift + ms_ix
            following_side_index = side_ix_shift + ((ms_ix + 1) % no_sides_in_cycle)

            side, side_bbox, mesh_ix, side_type = mesh_side
            mesh_last_point_ix = side.shape[0]

            spurrious_crossings.append((
                (absolute_side_index, mesh_last_point_ix), 
                (following_side_index, 0), 
                side[mesh_last_point_ix-1],
                1
            ))

            mesh_sides.append(mesh_side)
#             ax.plot(*side.T)
    #     marginal_side_splits.extend(spurrious_crossings)
        side_splits.extend(spurrious_crossings)

#     for mss in side_splits:
    #     if mss not in side_splits:
    #         side_splits.append(mss)

#         ax.scatter(*na(mss[2]).T, s=200)


    for side_id, (side, side_bbox, mesh_ix, side_type) in enumerate(mesh_sides):
        points_0 = side[:-1]
        points_1 = side[1:]

        cross_sections = []
        for rel_x_side_id, (x_side, x_side_bbox, x_mesh_ix, x_side_type) in enumerate(mesh_sides[side_id:]):
            x_side_id = rel_x_side_id + side_id

            points_x0 = x_side[:-1]
            points_x1 = x_side[1:]

            vectors = points_1 - points_0
            vectors_x = points_x1 - points_x0

            scaled_vec = points_x0[None, :] - points_0[:, None]
            vectors_cross = -numpy.cross(vectors_x[None, :], vectors[:, None], axisa=2, axisb=2)

            scaling = numpy.cross(scaled_vec, -vectors[:, None, :], axisa=2, axisb=2) / vectors_cross
            scaling_x = numpy.cross(scaled_vec, vectors_x[None, :, :], axisa=2, axisb=2) / vectors_cross

            fake_scale = 1
            cross_detection = numpy.logical_and(
                numpy.logical_and(
                    numpy.logical_and(0 <= scaling, scaling <= 1*fake_scale), 
                    numpy.logical_and(0 <= scaling_x, scaling_x <= 1*fake_scale)),
                True #vectors_cross != 0 # colinnear exclude
            )

            if side_id == x_side_id: # avoid aliasing 
                cross_detection = numpy.logical_and(cross_detection, numpy.tril(numpy.ones(cross_detection.shape), -1))

            cross_det_points_0, cross_det_vectors_x = numpy.where(cross_detection)

            cross_points = (points_0[cross_det_points_0] 
                + (vectors[cross_det_points_0] * scaling_x[cross_det_points_0, cross_det_vectors_x][:, None]))

            for crspix, cross_point in enumerate(cross_points):
                cross_point_1 = cross_det_points_0[crspix] + scaling[cross_det_points_0[crspix], cross_det_vectors_x[crspix]]
                cross_point_1x = cross_det_vectors_x[crspix] + scaling_x[cross_det_points_0[crspix], cross_det_vectors_x[crspix]]

                side_split = (side_id, cross_point_1)
                x_side_split = (x_side_id, cross_point_1x)
                direction = side_type * x_side_type
                side_splits.append((side_split, x_side_split, cross_point, direction))

#             ax.scatter(*cross_points.T, s=400, marker='o')


    # looping et crossection elimination
    split_indices = [[] for _ in range(len(mesh_sides))]
    for side_id, side_point_ix in [el for ss_a, ss_b, crosspoint, direction in side_splits for el in (ss_a, ss_b)]:
        split_indices[side_id].append(side_point_ix)
    split_indices = [sorted(list(side_split_indices)) for side_split_indices in split_indices]
    # print(split_indices)


    split_ranges = [[(sind[0],)*2]
                    + [(sind[ixix], sind[ixix+1]) for ixix in range(len(sind)-1)] 
                    + [(sind[-1],)*2] for sind in split_indices]
    # for x in split_ranges:
    #     print(x)

    split_ranges_bnd = [
        [[None] + [split_range[0] for split_range in srng[1:]]  for srng in split_ranges],
        [[split_range[1] for split_range in srng[:-1]] + [None] for srng in split_ranges] 
    ]
    bgn_crosspoint = {}
    end_crosspoint = {}
    # print(split_ranges_bnd)

    split_joints = []
    for side_split in side_splits:
        (side_id, split_bnd), (x_side_id, x_split_bnd), cross_point, direction = side_split
        side_split_low = split_ranges[side_id][split_ranges_bnd[1][side_id].index(split_bnd)]
        side_split_high = split_ranges[side_id][split_ranges_bnd[0][side_id].index(split_bnd)]
        x_side_split_low = split_ranges[x_side_id][split_ranges_bnd[1][x_side_id].index(x_split_bnd)]
        x_side_split_high = split_ranges[x_side_id][split_ranges_bnd[0][x_side_id].index(x_split_bnd)]

        if direction == 1: # assumption: crossing points does not repead for given segments
            split_joints.append(((side_id, side_split_low), (x_side_id, x_side_split_high), cross_point))
            split_joints.append(((side_id, side_split_high), (x_side_id, x_side_split_low), cross_point))

        elif direction == -1:
            split_joints.append(((side_id, side_split_low), (x_side_id, x_side_split_low), cross_point))
            split_joints.append(((side_id, side_split_high), (x_side_id, x_side_split_high), cross_point))

        end_crosspoint[(side_id, side_split_low)] = cross_point
        end_crosspoint[(x_side_id, x_side_split_low)] = cross_point
        bgn_crosspoint[(x_side_id, x_side_split_high)] = cross_point
        bgn_crosspoint[(side_id, side_split_high)] = cross_point



    side_cycles = []
    range_cycles = []
    for split_joint in split_joints: # probability of valid take = 1 - epsilon
        split, x_split, cross_point = split_joint
        new_cycle = [split_joint]
        for side_cycle in side_cycles[:]:
            cycle_flattened = [other_split for split_o, x_split_o, cross_point in side_cycle 
                               for other_split in  (split_o, x_split_o)]
    #         print(cycle_flattened)
            if split in cycle_flattened or x_split in cycle_flattened:
                new_cycle.extend(side_cycle)
                side_cycles.remove(side_cycle)
    #             print("dink")
        side_cycles.append(new_cycle)



    all_trackpoints = numpy.concatenate(track_pxs)[:, None]
    accepted_cycles = []
    # print(mesh_sides)

#     print(len(side_cycles))
    for rix, r in enumerate(side_cycles):

        range_cycle = set()
        for split, x_split, cross_point in r:
            side_id, side_range = split
            x_side_id, x_side_range = x_split

            side_ceil = (side_id, (ceil(side_range[0]), ceil(side_range[1])))
            x_side_ceil = (x_side_id, (ceil(x_side_range[0]), ceil(x_side_range[1])))

            if ceil(side_range[1]) - ceil(side_range[0]) > 0:
                range_cycle.add(split)
            if ceil(x_side_range[1]) - ceil(x_side_range[0]) > 0:
                range_cycle.add(x_split)

        acceptance_cycle = []
        for split in range_cycle:
            side_id, side_range = split
            side_range_ceil = (ceil(side_range[0]), ceil(side_range[1]))
            side = mesh_sides[side_id][0][side_range_ceil[0]:side_range_ceil[1]]

    #         if side_ceil in start_crosspoint:
            side = numpy.concatenate((bgn_crosspoint[split][None], side))
    #         if side_ceil in end_crosspoint:
            side = numpy.concatenate((side, end_crosspoint[split][None]))

            track_distances = numpy.sum((side - all_trackpoints) **2, axis=2)**.5
    #         print(side_ceil)
    #         print(numpy.min(track_distances))
            if numpy.any(track_distances < rad_min*0.6): #  ARBITRARY
    #             break
                pass
            else:
                acceptance_cycle.append(side)

#                 app = side
#                 ax.plot(*app.T, lw=4, c=cl2[rix%len(cl2)], ls='-')
#                 ax.plot(*app.T, lw=4, c=cl[rix%len(cl)], ls='--')
        else:
            accepted_cycles.extend(acceptance_cycle)
    
    
    total_side_points_no = (numpy.sum([side_t.shape[0] for side_t in accepted_cycles]) 
        + numpy.sum([track_t.shape[0] for track_t in track_pxs]))
    uni_side_points_no = int(total_side_points_no**.5)

    sides_uni = []

    for side in accepted_cycles:
        side_points_no = side.shape[0]

        splits_no = max(int(side_points_no/uni_side_points_no), 1)
#         print(side_points_no, 'aaa', uni_side_points_no)
        spec_uni_side_points_no = int(side_points_no/splits_no)

        for uni_ix in range(0, side_points_no, spec_uni_side_points_no):
            uni_side = side[uni_ix:(uni_ix+spec_uni_side_points_no+1)]
            uni_bbox = na((numpy.min(uni_side, axis=0), numpy.max(uni_side, axis=0)))

#             sides_uni.append((uni_side, uni_bbox, mesh_ix, side_type))
            sides_uni.append((uni_side, uni_bbox))

    track_uni = []
    directions_uni = []

    for six, side in enumerate(track_pxs):
        side_points_no = side.shape[0]

        splits_no = max(int(side_points_no/uni_side_points_no), 1)
#         print(side_points_no, 'aaa', uni_side_points_no)
        spec_uni_side_points_no = int(side_points_no/splits_no)

        for uni_ix in range(0, side_points_no, spec_uni_side_points_no):
            uni_side = side[uni_ix:(uni_ix+spec_uni_side_points_no+1)]
            uni_bbox = na((numpy.min(uni_side, axis=0), numpy.max(uni_side, axis=0)))

#             track_uni.append((uni_side, uni_bbox, mesh_ix, side_type))
            track_uni.append((uni_side, uni_bbox))
    
            uni_dirs = track_drs[six][uni_ix:(uni_ix+spec_uni_side_points_no+1)]
            uni_dbox = None
            directions_uni.append((uni_dirs, uni_dbox))

    return track_uni, sides_uni, directions_uni

In [ ]:
# drone utils

def get_circle_points(no):
    cd = numpy.linspace(0, 2*numpy.pi, no, endpoint=True)
    cc = na([numpy.sin(cd), numpy.cos(cd), 0*cd]).T
    return cc


def detect_crossings(side, x_side):
        points_0 = side[:-1]
        points_1 = side[1:]
        
        points_x0 = x_side[:-1]
        points_x1 = x_side[1:]

        vectors = points_1 - points_0
        vectors_x = points_x1 - points_x0

        scaled_vec = points_x0[None, :] - points_0[:, None]
        vectors_cross = -numpy.cross(vectors_x[None, :], vectors[:, None], axisa=2, axisb=2)

        scaling = numpy.cross(scaled_vec, -vectors[:, None, :], axisa=2, axisb=2) / vectors_cross
        scaling_x = numpy.cross(scaled_vec, vectors_x[None, :, :], axisa=2, axisb=2) / vectors_cross

        cross_detection = numpy.logical_and(
            numpy.logical_and(
                numpy.logical_and(0 <= scaling, scaling <= 1), 
                numpy.logical_and(0 <= scaling_x, scaling_x <= 1)),
            True #vectors_cross != 0 # colinnear exclude
        )

        cross_det_points_0, cross_det_vectors_x = numpy.where(cross_detection)

        cross_points = (points_0[cross_det_points_0] 
            + (vectors[cross_det_points_0] * scaling_x[cross_det_points_0, cross_det_vectors_x][:, None]))
#         print(cross_points)
        return cross_points

                
def drone_vis(rmx, collision_circles):
    rcc = (rmx @ get_circle_points(50).T).T

    circles = []
    rods = []
    for cpos, crad in collision_circles:
        apos = rmx @ cpos
        circles.append(na(apos[None]+rcc*crad)[:, :2])
        rods.append(na((apos, 0*apos))[:, :2])
        
    return circles, rods


def get_drone_rel_bbox(rmx, collision_circles):
    unitary_bboxes = []

    for cpos, crad in collision_circles:
        dist = numpy.sum(cpos**2)**.5
        ubbox = rmx @ (cpos * ((dist + crad * 2**.5)/dist))
        unitary_bboxes.append(ubbox)

    return na((numpy.min(unitary_bboxes, axis=0), numpy.max(unitary_bboxes, axis=0)))


def collisions_horz(rmx, collision_circles, pos0, side):
    rcc = (rmx @ get_circle_points(37).T).T

    cross_points = []
    for cpos, crad in collision_circles:
        apos = pos0 + rmx @ cpos
        circle = na(apos[None]+rcc*crad)[:, :2]
        cross_points.extend(detect_crossings(side, circle))
        
    return cross_points


def bbox_crossection(bbox1, bbox2):
    if (bbox1.shape != bbox2.shape or bbox1.shape != (2, 2) 
        or not numpy.all(bbox1[0]<=bbox1[1]) 
        or not numpy.all(bbox2[0]<=bbox2[1])):
#         print(bbox1.shape, bbox2.shape)
        raise ValueError()
    abox1 = na([[-bbox1[1][0], bbox1[0][1]], [-bbox1[0][0], bbox1[1][1]]])
    abox2 = na([[-bbox2[1][0], bbox2[0][1]], [-bbox2[0][0], bbox2[1][1]]])

    return (not (numpy.any(bbox1[1]<bbox2[0]) or numpy.any(bbox2[1]<bbox1[0]))
        or not (numpy.any(abox1[1]<abox2[0]) or numpy.any(abox2[1]<abox1[0])))
        
    
def bbox_vis(bbox):
    return na([bbox[0], (bbox[0][0], bbox[1][1]), bbox[1], (bbox[1][0], bbox[0][1]), bbox[0]])

In [ ]:
# track interactions

        
# init rays
class Rays:
    def __init__(self):
        horz_ray_dist = numpy.linspace(-numpy.pi/2, numpy.pi/2, num=15, endpoint=True)
        vert_ray_dist = numpy.linspace(-numpy.pi/3, numpy.pi/3, num=7, endpoint=True)
    #     ray_len = 400

        rays = numpy.zeros((len(horz_ray_dist), len(vert_ray_dist), 3))
        rays_indices = []
        for hix, horz in enumerate(horz_ray_dist):
            for vix, vert in enumerate(vert_ray_dist):
                ray_rmx = get_rmx([0, vert, horz])
                ray = ray_rmx @ na([1, 0, 0])
                ray_index = (hix, vix)

                rays[ray_index] = ray
                rays_indices.append(ray_index)
        
        self.rays = rays # norm = 1 !
        self.rays_indices = rays_indices
        self.ray_len = 800
        
# init derivative agent status
class AgentDevStatus:
    def __init__(self):
        # past relative
        self.checkpoints_reached = 0
        self.chp_segments_reached = 0 # sum of rewards from past segments
        self.next_checkpoint_ix = 0 # = current_track_ix
        self.crash_no = 0
        self.fall_no = 0

        # pos0 relative
        self.current_track_segment_ix = 0
        self.arbitrary_checkpoint_direction = na([0, 0])
        self.no_segment_change = 0
        self.velocity_sq = 0
        
        
def get_new_ags(track, ags, ds, is_crashed, is_fallen):
    new_ags = ags #AgentDevStatus()
    
    # checkpoint_reaching
    if numpy.sum((track.checkpoints[ags.next_checkpoint_ix] - ds.pos0[:2])**2) < (track.checkpoint_radius **2):
        new_ags.checkpoints_reached = ags.checkpoints_reached + 1 
        new_ags.chp_segments_reached = ags.chp_segments_reached + len(track.tracks[ags.next_checkpoint_ix][0])
        new_ags.next_checkpoint_ix = (ags.next_checkpoint_ix + 1) % len(track.checkpoints)
    else:
        new_ags.checkpoints_reached = ags.checkpoints_reached
        new_ags.chp_segments_reached = ags.chp_segments_reached
        new_ags.next_checkpoint_ix = ags.next_checkpoint_ix
    
    current_track = track.tracks[new_ags.next_checkpoint_ix][0]
    current_segment_ix = numpy.argmin(numpy.sum((current_track - ds.pos0[:2]) **2, axis=1))
    if current_segment_ix == ags.current_track_segment_ix:
        new_ags.no_segment_change = ags.no_segment_change + 1
    else:
        new_ags.no_segment_change = ags.no_segment_change
    new_ags.current_track_segment_ix = current_segment_ix
    new_ags.arbitrary_checkpoint_direction = track.checkpoints[new_ags.next_checkpoint_ix] - ds.pos0[:2]
    
    new_ags.crash_no += int(is_crashed)
    new_ags.fall_no += int(is_fallen)
    
    instant_dir = track.directions[new_ags.next_checkpoint_ix][0][new_ags.current_track_segment_ix]
    new_ags.velocity_sq = numpy.sum((numpy.max((ds.pos1[:2]*instant_dir, na([0, 0])))) ** 2)
#     print(is_crashed)
        
    return new_ags


def reward_calc(ags):
    # consts reward
    segment_reward = 250
    checkpoint_reward = 1e4
    crash_penalty = -1000
    fall_penalty = -300
    no_segment_change_penalty = 0
    no_velocity_advantage = 100

    state_reward =  ((ags.current_track_segment_ix + ags.chp_segments_reached) * segment_reward
        + ags.checkpoints_reached * checkpoint_reward
        + ags.crash_no * crash_penalty
        + ags.fall_no * fall_penalty
        + ags.no_segment_change * no_segment_change_penalty
        + ags.velocity_sq * no_velocity_advantage)
    
    return state_reward


def detect_collisions(pos0, rmx, collision_circles, track, current_track_bbox, vis=False):
    uav_bbox = get_drone_rel_bbox(rmx, collision_circles) + pos0[None]
    hbbox = uav_bbox[:, :2]
    vbbox = uav_bbox[:, 2]
#     ax.plot(*bbox_vis(hbbox).T, lw=5)

    active_bboxes = []

    # horizontal collision
    clp_horz = []
    for six, (side, bb) in enumerate(track.sides):
        if bbox_crossection(hbbox, bb):
            if vis:
                active_bboxes.append(bb)
            clp_horz.extend(collisions_horz(rmx, collision_circles, pos0, side))

    # vertical collision
    clp_vert = []
    if not track.vbnd[0] < vbbox[0]:
        clp_vert.append(na([*pos0[:2], track.vbnd[0]]))
    elif not vbbox[1] < track.vbnd[1]:
        clp_vert.append(na([*pos0[:2], track.vbnd[1]]))
    
    # drone lost
    clp_lost = []
    if not bbox_crossection(current_track_bbox, hbbox + track.checkpoint_radius * na([[-1, -1], [1, 1]])):
        clp_lost.append(pos0)
        
    return clp_horz, clp_vert, clp_lost, active_bboxes


def rays_take(pos0, rmx, rays, track, vis=False):
    rays_distances = numpy.zeros(rays.rays.shape[:2])
    rays_hitpoints = numpy.zeros(rays.rays.shape)
    for ray_index in rays.rays_indices:
        arbi_ray = rmx @ rays.rays[ray_index]
        ray = na([pos0, pos0 + arbi_ray * rays.ray_len])
        ray_bbox = na([numpy.min(ray, axis=0), numpy.max(ray, axis=0)])
        ray_hbbox = ray_bbox[:, :2]
        vh_ratio = arbi_ray[2]/(numpy.sum(arbi_ray[:2] **2) **.5) # vert-horz ratio of distances

        hit_points = [] # vis
        hits = []
        active_bboxes = []

        # horizontal hits 
        for six, (side, bb) in enumerate(track.sides):
            if bbox_crossection(ray_hbbox, bb):
                if vis:
                    active_bboxes.append(bb)
                
                horz_hits = na(detect_crossings(ray[:, :2], side))
                if not horz_hits.size:
                    continue

                for hhit in horz_hits:
                    vert_dist = numpy.sum((hhit-pos0[None, :2])**2) **.5*vh_ratio
                    vert_hit = pos0[2] + vert_dist
                    if vert_hit > track.vbnd[0] and vert_hit < track.vbnd[1]:
                        if vis:
                            hit_points.append((*hhit, vert_hit)) # vis
                        hits.append(numpy.sum((na([hhit[0], hhit[1], vert_hit])-pos0)**2)**.5)

        # vertical hits - these crossing walls are eliminated by taking least distance
        if abs(vh_ratio) > 1e-3:
            vert_hit = track.vbnd[0] if arbi_ray[2]<0 else track.vbnd[1]
            vert_diff = abs(pos0[2] - vert_hit)
            horz_diff = vert_diff / vh_ratio
            hits.append((vert_diff**2 + horz_diff**2)**.5)

            if vis:
                hhit = pos0[:2] + (arbi_ray[:2]/arbi_ray[2]) * horz_diff
                hit_points.append((*hhit, vert_hit))

        if numpy.any(na(hits) < rays.ray_len):
            rays_distances[ray_index] = numpy.min(hits)
            if vis:
                rays_hitpoints[ray_index] = na(hit_points[numpy.argmin(hits)])
        else:
            rays_distances[ray_index] = rays.ray_len
    
    return rays_distances, rays_hitpoints, active_bboxes



In [ ]:
# drone physics

class DroneState:
    def __init__(self):
        self.pos0 = numpy.zeros((3,))
        self.pos1 = numpy.zeros((3,))
        self.pos2 = numpy.zeros((3,))
        self.pos3 = numpy.zeros((3,))

        self.rot0 = numpy.zeros((3,))
        self.rot1 = numpy.zeros((3,))
        self.rot2 = numpy.zeros((3,))
        self.rot3 = numpy.zeros((3,))
            
def get_rmx(rot0):
    r_cos = numpy.cos(rot0)
    r_sin = numpy.sin(rot0)
    rmx_comp = na([
        [[ 1,         0,         0],
         [ 0,         r_cos[0], -r_sin[0]],
         [ 0,         r_sin[0],  r_cos[0]]],
        [[ r_cos[1],  0,         r_sin[1]],
         [ 0,         1,         0],
         [-r_sin[1],  0,         r_cos[1]]],
        [[ r_cos[2], -r_sin[2],  0],
         [ r_sin[2],  r_cos[2],  0],
         [ 0,         0,         1]]
        ])
    rmx = rmx_comp[0] @ rmx_comp[1] @ rmx_comp[2]
    return rmx


def integrate2(y0, y1, y2, y3, ny2, dt):    
    ny3 = (ny2 - y2)/dt
    ny2 = ny2
    ny1 = y1 + (y2+ny2)/2*dt
    ny0 = y0 + (y1+ny1)/2*dt
    
    return ny0, ny1, ny2, ny3

In [ ]:
# draw track
def draw_track(ax, track):
    sides_conn = []
    for side, bb in track.sides:
        sides_conn.extend(side)
    sides_conn_min = numpy.min(sides_conn, axis=(0, 1)) - track.checkpoint_radius
    sides_conn_max = numpy.max(sides_conn, axis=(0, 1)) + track.checkpoint_radius
    ax.set(xlim=(sides_conn_min, sides_conn_max), ylim=(sides_conn_min, sides_conn_max))
    
    for tk, bb in track.tracks:
        ax.plot(*tk.T, c='gray')
        ax.plot(*bbox_vis(bb).T, lw=.5, c='lightgray')
    for side, bb in track.sides:
        ax.plot(*side.T, lw=3, c='black')
        ax.plot(*bbox_vis(bb).T, lw=.5, c='lightgray')

    cc = get_circle_points(50)[:, :2]
    for ix, cpos in enumerate(track.checkpoints):
        ax.plot(*(cpos[None] + cc*track.checkpoint_radius).T, c='gray')
        ax.scatter(*cpos.T, marker='${}$'.format(ix), s=500, c='gray')

class VaryingDraws():
    def __init__(self, ax):
        self.rotors = [ax.plot([], [], lw=2, c='black')[0] for _ in range(4)]
        self.rods = [ax.plot([], [], lw=2, c='gray')[0] for _ in range(4)]
        
        self.rays_horz_hit = ax.scatter([], [], c='blue')
        self.rays_vert_hit = ax.scatter([], [], c='cyan')
        
        self.clp_horz = ax.scatter([], [], c='red', s=1000)
        self.clp_vert = ax.scatter([], [], c='red', s=1000)
        self.clp_lost = ax.scatter([], [], c='red', s=1000)
        
        self.bboxes_active_shits = [ax.plot([], [], lw=.5, c='cyan')[0] for _ in range(32)]
        self.bboxes_active_sides = [ax.plot([], [], lw=.8 ,c='red')[0] for _ in range(8)]
        self.bbox_active_track = ax.plot([], [], lw=.5, c='lime')[0]
        self.bbox_uav = ax.plot([], [], lw=.8, c='red')[0]
        self.active_checkpoint = ax.plot([], [], lw=.5, c='lime')[0]
        
        self.frame_counter = 0
        text_pos = na([-300, -100])
        text_spread = 15
        self.frame_viewer = ax.text(*(text_pos + na([0, 0*text_spread])), '0')
        self.reward_viewer = ax.text(*(text_pos + na([0, 1*text_spread])), '0')
        self.height_viewer = ax.text(*(text_pos + na([0, 2*text_spread])), '0')

def draw_varying(vd, track, ds, dp, rmx, rays, rays_hitpoints, rays_bboxes, 
                 clp_bboxes, clp_horz, clp_vert, clp_lost, state_reward, ags):
#     print(rays_hitpoints)
    # DRAWING
    # draw uav
    c, r = drone_vis(rmx, dp.collision_circles)
    cc = get_circle_points(50)[:, :2]
    for ix in range(4):
        vd.rotors[ix].set_data(*(ds.pos0[None, :2]+c[ix]).T)
        vd.rods[ix].set_data(*(ds.pos0[None, :2]+r[ix]).T)
    # draw rays
    rays_horz_hit = [rays_hitpoints[ray_index][:2] for ray_index in rays.rays_indices 
        if (rays_hitpoints[ray_index][2] > track.vbnd[0] and rays_hitpoints[ray_index][2] < track.vbnd[1])]
    if rays_horz_hit:
        vd.rays_horz_hit.set_offsets(rays_horz_hit)
    else:
        vd.rays_horz_hit.set_offsets(numpy.zeros((0, 2)))
    rays_vert_hit = [rays_hitpoints[ray_index][:2] for ray_index in rays.rays_indices 
        if not (rays_hitpoints[ray_index][2] > track.vbnd[0] and rays_hitpoints[ray_index][2] < track.vbnd[1])]
    if rays_vert_hit:
        vd.rays_vert_hit.set_offsets(rays_vert_hit)
    else:
        vd.rays_vert_hit.set_offsets(numpy.zeros((0, 2)))
    # draw collisions
    if clp_horz:
        vd.clp_horz.set_offsets(na(clp_horz)[:, :2])
    else:
        vd.clp_horz.set_offsets(numpy.zeros((0, 2)))
    if clp_vert:
        vd.clp_vert.set_offsets(na(clp_vert)[:, :2])
    else:
        vd.clp_vert.set_offsets(numpy.zeros((0, 2)))
    if clp_lost:
        vd.clp_lost.set_offsets(na(clp_lost)[:, :2])
    else:
        vd.clp_lost.set_offsets(numpy.zeros((0, 2)))
    # draw bboxes
    for bix in range(min(len(vd.bboxes_active_shits), len(rays_bboxes))):
        vd.bboxes_active_shits[bix].set_data(*bbox_vis(rays_bboxes[bix]).T)
    for bix in range(len(vd.bboxes_active_shits), len(rays_bboxes)):
        vd.bboxes_active_shits[bix].set_data([], [])
    for bix in range(min(len(vd.bboxes_active_sides), len(clp_bboxes))):
        vd.bboxes_active_sides[bix].set_data(*bbox_vis(clp_bboxes[bix]).T)
    for bix in range(len(vd.bboxes_active_sides), len(clp_bboxes)):
        vd.bboxes_active_sides[bix].set_data([], [])
    vd.bbox_active_track.set_data(*bbox_vis(track.tracks[ags.next_checkpoint_ix][1]).T)
    vd.active_checkpoint.set_data(*(cc*track.checkpoint_radius+track.checkpoints[ags.next_checkpoint_ix]).T)
    vd.bbox_uav.set_data(*(bbox_vis(get_drone_rel_bbox(rmx, dp.collision_circles)[:, :2]) + ds.pos0[None][:, :2]).T)
    
    vd.frame_counter +=1
    vd.frame_viewer.set_text(str(vd.frame_counter))
    vd.reward_viewer.set_text(str(state_reward))
    vd.height_viewer.set_text(str(ds.pos0[2]))


In [ ]:
# init drone
class DroneParams:
    def __init__(self):
        self.gravitional_acc = 9.81
        
        #drone params
        mass_centre = .5
        mass_motor  = .125
        yaw_torque_ratio = .2
        rotor_radius = 2
        motor_pos_radius = 2

        centre_position = na([0, 0, 0])
        motor_position = na([[1, 1, 0], [-1, 1, 0], [-1, -1, 0], [1, -1, 0]]) * motor_pos_radius

        # drone-relative
        masses = [(mpos, mass_motor) for mpos in motor_position] + [(centre_position, mass_centre)]
        mass_total = numpy.sum([mass for pos, mass in masses])

        inertia_pos = numpy.ones((3,)) * mass_total
        inertia_rot = na([numpy.sum([numpy.sum(numpy.cross(mpos, numpy.diag((1,)*3)[dim]) **2) * mass 
                                     for mpos, mass in masses]) for dim in range(3)])

        collision_circles = [(mpos, rotor_radius) for mpos in motor_position]

        motor_thrust = self.gravitional_acc * mass_total  * na([0, 0, 1]) 
        motor_torque = yaw_torque_ratio * motor_thrust

        thrusts = [(mpos, motor_thrust) for mpos in motor_position]
        torques = [(mpos, motor_torque * (-1)**ix) for ix, mpos in enumerate(motor_position)]
        
        self.inertia_pos = inertia_pos
        self.inertia_rot = inertia_rot
        
        self.masses = masses
        self.thrusts = thrusts
        self.torques = torques
        
        self.collision_circles = collision_circles
        self.max_iteration_dpos = motor_pos_radius

# init track
class Track:
    def __init__(self, seed):
        track_radius = 10
        self.checkpoint_radius = track_radius*1.5
        vbnd = (0, 100)

        tracks, sides, directions = race_gen(seed, track_radius)
        checkpoints = [track[0][-1] for track in tracks]

        self.tracks = tracks
        self.sides = sides
        self.directions = directions
        self.checkpoints = checkpoints
        self.vbnd = vbnd
        
        track_conn = []
        for track, bb in tracks:
            track_conn.extend(track)
        self.total_len = len(track_conn)
        
# INTERACTIONS
def environment_interactions(track, ds, dp, rays, ags, vd, vis):
    rmx = get_rmx(ds.rot0)

    # collision_detection
    clp_horz, clp_vert, clp_lost, clp_bboxes = detect_collisions(ds.pos0, rmx, dp.collision_circles, track, 
        current_track_bbox=track.tracks[ags.next_checkpoint_ix][1], vis=True)

    is_crashed = False
    is_fallen = False
    # collision correction
    if clp_horz or clp_vert or clp_lost:
        
        correction_direction = track.directions[ags.next_checkpoint_ix][0][0]

        ds.pos0[2] = track.vbnd[0]+(track.vbnd[1]-track.vbnd[0])*1e-1# fall_down
        ds.rot0 = na([0, 0, numpy.arctan2(*correction_direction[::-1])]) # restore correct rotation
        rmx = get_rmx(ds.rot0)

        ds.pos1 = numpy.zeros(3)
        ds.pos2 = numpy.zeros(3)
        ds.pos3 = numpy.zeros(3)
        ds.rot1 = numpy.zeros(3)
        ds.rot2 = numpy.zeros(3)
        ds.rot3 = numpy.zeros(3)

        if clp_horz or clp_lost:
            is_crashed = True
            correction_position = track.tracks[ags.next_checkpoint_ix][0][0]

            ds.pos0[:2] = correction_position # return to safezone
        else:
            is_fallen = True

    # rays readings
    rays_distances, rays_hitpoints, rays_bboxes = rays_take(ds.pos0, rmx, rays, track, vis=True)

    # reward estimation
    ags = get_new_ags(track, ags, ds, is_crashed, is_fallen)
    state_reward = reward_calc(ags)
    instant_dir = track.directions[ags.next_checkpoint_ix][0][ags.current_track_segment_ix]

    if vis:
        draw_varying(vd, track, ds, dp, rmx, rays, rays_hitpoints, rays_bboxes, 
                     clp_bboxes, clp_horz, clp_vert, clp_lost, state_reward, ags)
    
#     print(ags.next_checkpoint_ix)
    return rays_distances, state_reward, ags, instant_dir

def uav_dynamics(rmx, dp, steering):
    gravity = dp.gravitional_acc * na([0, 0, -1])
    
    forces_mass = [mass * gravity for mpos, mass in dp.masses]
    forces_thrust = [steering[ix] * (rmx @ thrust) for ix, (mpos, thrust) in enumerate(dp.thrusts)]

    torques_mass = [mass * numpy.cross(gravity, rmx @ mpos) for mpos, mass in dp.masses]
    torques_thrust = [steering[ix] * numpy.cross(rmx @ thrust, rmx @ mpos) 
                      for ix, (mpos, thrust) in enumerate(dp.thrusts)]
    torques_torque = [steering[ix] * rmx @ torque for ix, (mpos, torque) in enumerate(dp.torques)]

    resultant_force = numpy.sum(numpy.concatenate((forces_mass, forces_thrust)), axis=0)
    resultant_torque = numpy.sum(numpy.concatenate((torques_mass, torques_thrust, torques_torque)), axis=0)

    pos2 = resultant_force  / (rmx @ dp.inertia_pos)
    rot2 = resultant_torque / (rmx @ dp.inertia_rot)
    
    return pos2, rot2


class Environment:
    def __init__(self, seed, vis=False, debug=False):
        # episode globals
        self.vis = vis
        self.track = Track(seed) # 28
        self.rays = Rays()
        self.ags = AgentDevStatus()
        self.dp = DroneParams()
        self.ds = DroneState()

        # init uav position
        self.ds.pos0 = na([self.track.tracks[0][0][0][0], self.track.tracks[0][0][0][1], 
                      self.track.vbnd[0]+(self.track.vbnd[1]-self.track.vbnd[0])*1e-2])
        self.ds.rot0 = na([0, 0, numpy.arctan2(*self.track.directions[0][0][0][::-1])])
        
        #DEBUG
        if debug:
            self.ds.pos1 = na([-10, 10, 0])

        # draw canvas 
        if vis:
            self.fig, ax = plt.subplots(1, 1, figsize=(10,)*2)
            draw_track(ax, self.track)
            self.vd = VaryingDraws(ax)
        else:
            self.vd = None

        # integral properties
        self.max_dt = .1
        self.max_dt_dist = 3
        self.max_dt_revl = .05 * numpy.pi
        
    def iteration(self, action):
        track = self.track
        vis = self.vis
        rays = self.rays
        ags = self.ags
        dp = self.dp
        ds = self.ds

        if vis:
            fig = self.fig
        vd = self.vd
        max_dt = self.max_dt
        max_dt_dist = self.max_dt_dist
        max_dt_revl = self.max_dt_revl

        
        steering = na(action)
        rmx = get_rmx(ds.rot0)
        pos2, rot2 = uav_dynamics(rmx, dp, steering)

        uav_velocity = numpy.sum(ds.pos1 ** 2) **.5 + 1e-9
        uav_revolution = numpy.sum(ds.rot1 ** 2) **.5 + 1e-9
        dt = min(max_dt, min(max_dt_dist/uav_velocity, max_dt_revl/uav_revolution))

        ds.pos0, ds.pos1, ds.pos2, ds.pos3 = integrate2(ds.pos0, ds.pos1, ds.pos2, ds.pos3, pos2, dt)
        ds.rot0, ds.rot1, ds.rot2, ds.rot3 = integrate2(ds.rot0, ds.rot1, ds.rot2, ds.rot3, rot2, dt)

        rays_distances, state_reward, agent_dev_status, instant_direction \
            = environment_interactions(track, ds, dp, rays, ags, vd, vis)

        state = (rays_distances, 
                 na([[ds.pos0, ds.pos1, ds.pos2, ds.pos3], [ds.rot0, ds.rot1, ds.rot2, ds.rot3]]),
                 ags.arbitrary_checkpoint_direction,
                 instant_direction
                )
        reward = state_reward
        done = (ags.chp_segments_reached + ags.current_track_segment_ix) / track.total_len
        
        if vis:
            fig.canvas.draw()

        return state, reward, done


In [ ]:
%matplotlib notebook

n=500
# steering = [.25, .25, .25, .25]  # share of thrust realtively

envi = Environment(35, True)
state, reward, done = envi.iteration([.25, .25, .25, .25])


for _ in range(n):
    action = policy_net.get_action(state).detach()
    state, reward, done = envi.iteration(action)
    

In [ ]:
class ReplayBuffer:
    def __init__(self, capacity):
        self.capacity = capacity
        self.buffer = []
        self.position = 0
    
    def push(self, state, action, reward, next_state, done):
        if len(self.buffer) < self.capacity:
            self.buffer.append(None)
        self.buffer[self.position] = (state, action, reward, next_state, done)
        self.position = (self.position + 1) % self.capacity
    
    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        state, action, reward, next_state, done = map(np.stack, zip(*batch))
        return state, action, reward, next_state, done
    
    def __len__(self):
        return len(self.buffer)

    
class ValueNetwork(nn.Module):
    def __init__(self, state_dim, hidden_dim, init_w=3e-3):
        super(ValueNetwork, self).__init__()
        
        self.linear1 = nn.Linear(state_dim, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, hidden_dim)
        self.linear3 = nn.Linear(hidden_dim, hidden_dim)
        self.linear4 = nn.Linear(hidden_dim, hidden_dim)
        self.linear5 = nn.Linear(hidden_dim, 1)
        
        self.linear5.weight.data.uniform_(-init_w, init_w)
        self.linear5.bias.data.uniform_(-init_w, init_w)
        
    def forward(self, state):
#         ray, pos, chp = state

#         ray = torch.flatten(ray)
#         pos = torch.flatten(pos)
#         chp = torch.flatten(chp)
#         x = torch.cat([ray, pos, chp])
        x = state
        
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = F.relu(self.linear3(x))
        x = F.relu(self.linear4(x))
        x = self.linear5(x)
        return x
        
        
class SoftQNetwork(nn.Module):
    def __init__(self, num_inputs, num_actions, hidden_size, init_w=3e-3):
        super(SoftQNetwork, self).__init__()
        
        self.linear1 = nn.Linear(num_inputs + num_actions, hidden_size)
        self.linear2 = nn.Linear(hidden_size, hidden_size)
        self.linear3 = nn.Linear(hidden_size, hidden_size)
        self.linear4 = nn.Linear(hidden_size, hidden_size)
        self.linear5 = nn.Linear(hidden_size, num_actions)
        
        self.linear5.weight.data.uniform_(-init_w, init_w)
        self.linear5.bias.data.uniform_(-init_w, init_w)
        
    def forward(self, state, action):
#         print(state)
#         ray, pos, chp = state
        
#         ray = torch.flatten(ray)
#         pos = torch.flatten(pos)
#         chp = torch.flatten(chp)
#         act = torch.flatten(action)
#         x = torch.cat([ray, pos, chp, act])
        x = torch.cat([state, action], 1)
        
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = F.relu(self.linear3(x))
        x = F.relu(self.linear4(x))
        x = self.linear5(x)
        return x
        
        
class PolicyNetwork(nn.Module):
    def __init__(self, num_inputs, num_actions, hidden_size, init_w=3e-3, log_std_min=-20, log_std_max=2):
        super(PolicyNetwork, self).__init__()
        
        self.log_std_min = log_std_min
        self.log_std_max = log_std_max
        
        self.conv1   = nn.Conv2d(1, 8, kernel_size=3, stride=1, padding=1)
        self.pool1   = nn.MaxPool2d(kernel_size=2, stride=1)
#         self.conv2   = nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1)
#         self.pool2   = nn.MaxPool2d(kernel_size=2, stride=1)
        
        self.linear1 = nn.Linear(700, hidden_size) # too lazy to compute otput from above layers
        self.linear2 = nn.Linear(hidden_size, hidden_size)
        self.linear3 = nn.Linear(hidden_size, hidden_size)
        self.linear4 = nn.Linear(hidden_size, hidden_size)
        self.linear5 = nn.Linear(hidden_size, hidden_size)
        
        self.mean_linear = nn.Linear(hidden_size, num_actions)
        self.mean_linear.weight.data.uniform_(-init_w, init_w)
        self.mean_linear.bias.data.uniform_(-init_w, init_w)
        
        self.log_std_linear = nn.Linear(hidden_size, num_actions)
        self.log_std_linear.weight.data.uniform_(-init_w, init_w)
        self.log_std_linear.bias.data.uniform_(-init_w, init_w)
        
    def forward(self, state):
#         print(state)
        ray, pos, chp, idr= state
        
        ray = torch.FloatTensor(ray[None, None])
        pos = torch.FloatTensor(pos)
        chp = torch.FloatTensor(chp)
        idr = torch.FloatTensor(idr)
        
        ray = self.conv1(ray)
        ray = F.relu(ray)
        ray = self.pool1(ray)
#         ray = self.conv2(ray)
#         ray = F.relu(ray)
#         ray = self.pool2(ray)
        
        ray = torch.flatten(ray)
        pos = torch.flatten(pos)
        chp = torch.flatten(chp)
        idr = torch.flatten(idr)
        x = torch.cat([ray, pos, chp, idr])
#         print(x)

        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = F.relu(self.linear3(x))
        x = F.relu(self.linear4(x))
        x = F.relu(self.linear5(x))
        
        mean    = self.mean_linear(x)
        log_std = self.log_std_linear(x)
        log_std = torch.clamp(log_std, self.log_std_min, self.log_std_max)
        
        return mean, log_std
    
    def evaluate(self, state, epsilon=1e-6):
        mean_t = []
        log_std_t = []
        for stat in state:
            mean, log_std = self.forward(stat)
            mean_t.append(mean)
            log_std_t.append(log_std)
            
        mean = torch.stack(mean_t)
        log_std = torch.stack(log_std_t)
        std = log_std.exp()
        
        normal = Normal(0, 1)
        z      = normal.sample()
        action = torch.tanh(mean+ std*z.to(device))
        log_prob = Normal(mean, std).log_prob(mean+ std*z.to(device)) - torch.log(1 - action.pow(2) + epsilon)
        return action, log_prob, z, mean, log_std
        
    
    def get_action(self, state):
#         state = torch.FloatTensor(state).unsqueeze(0).to(device) # wtf? todo
        mean, log_std = self.forward(state)
        std = log_std.exp()
#         std = log_std
    
        normal = Normal(0, 1)
        z      = normal.sample().to(device)
        action = torch.tanh(mean + std*z)
        
        action  = action.cpu()#.detach().cpu().numpy()
        return action#[0]

    
def plot(frame_idx, rewards):
    clear_output(True)
    plt.figure(figsize=(20,5))
    plt.subplot(131)
    plt.title('frame %s. reward: %s' % (frame_idx, rewards[-1]))
    plt.plot(rewards)
    plt.show()

    
def update(batch_size,gamma=0.94,soft_tau=.08,):
    state, action, reward, next_state, done = replay_buffer.sample(batch_size)
    
    policy_state = state
    state = [numpy.concatenate([x.reshape(-1) for x in tupl]) for tupl in state]
    next_state = [numpy.concatenate([x.reshape(-1) for x in tupl]) for tupl in next_state]


    state      = torch.FloatTensor(state).to(device)
    next_state = torch.FloatTensor(next_state).to(device)
    action     = torch.FloatTensor(action).to(device)
    reward     = torch.FloatTensor(reward).unsqueeze(1).to(device)
    done       = torch.FloatTensor(np.float32(done)).unsqueeze(1).to(device)

    predicted_q_value1 = soft_q_net1(state, action)
    predicted_q_value2 = soft_q_net2(state, action)
    predicted_value    = value_net(state)
    new_action, log_prob, epsilon, mean, log_std = policy_net.evaluate(policy_state)

    
# Training Q Function
    target_value = target_value_net(next_state)
    target_q_value = reward + (1 - done) * gamma * target_value
    q_value_loss1 = soft_q_criterion1(predicted_q_value1, target_q_value.detach())
    q_value_loss2 = soft_q_criterion2(predicted_q_value2, target_q_value.detach())


    soft_q_optimizer1.zero_grad()
    q_value_loss1.backward()
    soft_q_optimizer1.step()
    soft_q_optimizer2.zero_grad()
    q_value_loss2.backward()
    soft_q_optimizer2.step()    
# Training Value Function
    predicted_new_q_value = torch.min(soft_q_net1(state, new_action), soft_q_net2(state, new_action))
    target_value_func = predicted_new_q_value - log_prob
#     print(predicted_value, target_value_func)
    value_loss = value_criterion(predicted_value, target_value_func.detach())

    
    value_optimizer.zero_grad()
    value_loss.backward()
    value_optimizer.step()
# Training Policy Function
    policy_loss = (log_prob - predicted_new_q_value).mean()

    policy_optimizer.zero_grad()
    policy_loss.backward()
    policy_optimizer.step()
    
    
    for target_param, param in zip(target_value_net.parameters(), value_net.parameters()):
        target_param.data.copy_(
            target_param.data * (1.0 - soft_tau) + param.data * soft_tau
        )

In [ ]:
action_dim = 4
state_dim  = numpy.sum([numpy.prod(x) for x in ((15, 7), (2, 4, 3), 2, 2)]) # formally envi-dependent
print(state_dim)
hidden_dim = 512

value_net        = ValueNetwork(state_dim, hidden_dim).to(device)
target_value_net = ValueNetwork(state_dim, hidden_dim).to(device)

soft_q_net1 = SoftQNetwork(state_dim, action_dim, hidden_dim).to(device)
soft_q_net2 = SoftQNetwork(state_dim, action_dim, hidden_dim).to(device)
policy_net = PolicyNetwork(state_dim, action_dim, hidden_dim).to(device)

for target_param, param in zip(target_value_net.parameters(), value_net.parameters()):
    target_param.data.copy_(param.data)

value_criterion  = nn.MSELoss()
soft_q_criterion1 = nn.MSELoss()
soft_q_criterion2 = nn.MSELoss()

value_lr  = 3e-4
soft_q_lr = 3e-4
policy_lr = 3e-4

value_optimizer  = optim.Adam(value_net.parameters(), lr=value_lr)
soft_q_optimizer1 = optim.Adam(soft_q_net1.parameters(), lr=soft_q_lr)
soft_q_optimizer2 = optim.Adam(soft_q_net2.parameters(), lr=soft_q_lr)
policy_optimizer = optim.Adam(policy_net.parameters(), lr=policy_lr)

PATH = "/home/kiciu/saved_models/"
save_key = datetime.datetime.now().strftime("%Y%m%d%H%M")
frame_idx   = 0


In [ ]:
PATH = ""
save_key = "202201081954"
frame_idx = 2500 #135000
marker = "_{}_{}.pth".format(save_key, frame_idx)

value_net        = torch.load(PATH + "value_net" + marker)
target_value_net = torch.load(PATH + "target_value_net" + marker)
soft_q_net1      = torch.load(PATH + "soft_q_net1" + marker)
soft_q_net2      = torch.load(PATH + "soft_q_net2" + marker)
policy_net       = torch.load(PATH + "policy_net" + marker)

In [ ]:
# HY-PA
replay_buffer_size = 10000

max_frames  = 4000e3
max_steps   = 2500
save_frame  = 2500
batch_size  = 256


In [ ]:
%matplotlib inline

verified_seeds = [
     0,  1,  3,  4,  6,  7,  9, 10, 11, 12, 
    13, 15, 17, 18, 19, 20, 21, 23, 24, 25, 
    28, 29, 31, 32, 34, 35, 37, 38, 40, 42,
    44, 45, 46, 47, 48, 51, 52, 53, 54, 55,
    56, 61, 62, 63, 65, 66, 67, 68, 69, 71,
] # risky 22, 33, 36
random.shuffle(verified_seeds)
rewards = [] #  here must be empty

while frame_idx < max_frames:
    if not (frame_idx % max_steps):
        if rewards:
            marker = "_{}_{}_seed{}.pth".format(save_key, frame_idx, seed)
            with open(PATH + "rewards" + marker, "w") as f:
                for s in rewards[int(-save_frame):]:
                    f.write(str(s) +"\n")
            print("rewards saved with marker: {}".format(marker))
        
        seed = verified_seeds[int(frame_idx//max_steps) % len(verified_seeds)]
        print(seed)
        envi = Environment(seed)
        replay_buffer = ReplayBuffer(replay_buffer_size)
        rewards     = []
        
        state, reward, done = envi.iteration(na([.25, .25, .25, .25])) # thurst/gravity ratio
        episode_reward = reward
        
    action = policy_net.get_action(state).detach()
    next_state, reward, done = envi.iteration(action.numpy())

    replay_buffer.push(state, action, reward, next_state, done)
        
    state = next_state
    episode_reward = reward
    
    if not(frame_idx % save_frame):
        marker = "_{}_{}.pth".format(save_key, frame_idx)
        torch.save(value_net, PATH + "value_net" + marker)
        torch.save(target_value_net, PATH + "target_value_net" + marker)
        torch.save(soft_q_net1, PATH + "soft_q_net1" + marker)
        torch.save(soft_q_net2, PATH + "soft_q_net2" + marker)
        torch.save(policy_net, PATH + "policy_net" + marker)
        print("models saved with marker: {}".format(marker))
                
    frame_idx += 1
        
    if len(replay_buffer) > batch_size:
        update(batch_size)

    if frame_idx % 100 == 0:
        plot(frame_idx, rewards)
    
    if done >= 3:
        break     
#     print(frame_idx)
    rewards.append(episode_reward)